<a href="https://colab.research.google.com/github/pacificblue/Data-Science-Tutorials/blob/master/7%20CV/Image_Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.4.1


# The Keras functional API

 ## Coding tutorials
 #### [1. Accessing model layers](#coding_tutorial_3)
 #### [2. Freezing layers](#coding_tutorial_4)

***
<a id="coding_tutorial_3"></a>
## Accessing model layers

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### Load the pre-trained model

In this section, we aim to demonstrate accessing layer attributes within a model.

Let's get started by loading the `VGG19` pre-trained model from the `keras.applications` library, which is a very deep network trained on more than a million images from the ImageNet database. The network is trained to classify images into 1000 object categories.

In [ ]:
! pip install pypac

In [ ]:
# Load the VGG19 model
from tensorflow.keras.applications import VGG19

vgg_model = VGG19(weights='imagenet')

In [ ]:
# Get the inputs, layers and display the summary
vgg_input = vgg_model.input
vgg_layers = vgg_model.layers
vgg_model.summary()

#### Build a model to access the layer outputs

In [ ]:
from tensorflow.keras.models import Model

In [ ]:
# Build a model that returns the layer outputs
layer_outputs = [layer.output for layer in vgg_layers]
features = Model(inputs=vgg_input, outputs=layer_outputs)

In [ ]:
# Plot the model
plot_model(features, 'vgg19_model.png', show_shapes=True)

In [ ]:
# Test the model on a random input
img = np.random.random((1, 224, 224, 3)).astype('float32')
extracted_features = features(img)

#### Load the 'cool cat' picture

In Zambia’s South Luangwa National Park, a photographer had been watching a pride of lions while they slept off a feast from a buffalo kill. When this female walked away, he anticipated that she might be going for a drink and so he positioned his vehicle on the opposite side of the waterhole. The `cool cat` picture is one of the highly commended 2018 Image from Wildlife Photographer of the Year.

In [ ]:
# Display the original image
import IPython.display as display
from PIL import Image

display.display(Image.open('data/cool_cat.jpg'))

FileNotFoundError: ignored

#### Visualise network features from the input image

In [ ]:
# Preprocess the image

from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image

img_path = 'data/cool_cat.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [ ]:
# Extract the features
extracted_features = features(x)

In [ ]:
# Visualise the input channels

imgs = f1[0, :, :]
plt.figure(figsize=(15, 15))

for n in range(3):
    ax = plt.subplot(1, 3, n + 1)
    plt.imshow(imgs[:, :, n])
    plt.axis('off')
plt.subplots_adjust(wspace=0.01, hspace=0.01)

In [ ]:
# Visualise some features in the first hidden layer
f2 = extracted_features[1]
print('f2.shape: ', f2.shape)

imgs = f2[0, :, :]
plt.figure(figsize=(15, 15))

for n in range(16):
    ax = plt.subplot(4, 4, n + 1)
    plt.imshow(imgs[:, :, n])
    plt.axis('off')
plt.subplots_adjust(wspace=0.01, hspace=0.01)

In [ ]:
# Build a model to extract features by layer name
extracted_features_block1_pool = Model(inputs=features.input, 
                                       outputs=features.get_layer('block1_pool').output)
block1_pool_features = extracted_features_block1_pool.predict(x)

In [ ]:
# Visualise some features from the extracted layer output
f3 = block1_pool_features
print('f3.shape: ', f3.shape)

imgs = f3[0, :, :]
plt.figure(figsize=(15, 15))

for n in range(16):
    ax = plt.subplot(4, 4, n + 1)
    plt.imshow(imgs[:, :, n])
    plt.axis('off')
plt.subplots_adjust(wspace=0.01, hspace=0.01)

In [ ]:
# Build a model to extract features by layer name
extracted_features_block5_conv4 = Model(inputs=features.input, 
                                       outputs=features.get_layer('block5_conv4').output)
block5_conv4_features = extracted_features_block5_conv4.predict(x)

# Visualize some features in the first hidden layer
f4 = block5_conv4_features
print('f4.shape: ', f4.shape)

imgs = f4[0, :, :]
plt.figure(figsize=(15, 15))

for n in range(16):
    ax = plt.subplot(4, 4, n + 1)
    plt.imshow(imgs[:, :, n])
    plt.axis('off')
plt.subplots_adjust(wspace=0.01, hspace=0.01)

***
<a id="coding_tutorial_4"></a>
## Freezing layers

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

#### Build the model

In [ ]:
# Build a small Sequential model

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

model = Sequential([
    layers.Dense(4, input_shape=(4,), activation='relu', kernel_initializer='random_uniform',
                 bias_initializer='ones'),
    layers.Dense(2, activation='relu', kernel_initializer='lecun_normal', bias_initializer='ones'),
    layers.Dense(4, activation='softmax'),
])

In [ ]:
# Display the model summary

model.summary()

#### Examine the weight matrix variation over training

In [ ]:
# Retrieve the weights and biases
def  get_weights(model):
    return [e.weights[0].numpy() for e in model.layers]

def get_biases(model):
    return [e.bias.numpy() for e in model.layers]

def plot_delta_weights(W0_lyaers, W1_layers, b0_layers, b1_layers):
    # Plot the variation
    plt.figure(figsize=(8, 8))
    
    for n in range(3):
        delta_l = W1_layers[n] - W0_layers[n]
        print("Layer " + str(n) + ': Bias variation: ', np.linalg.norm(b1_layers[n] - b0_layers[n]))
        ax = plt.subplot(1, 3, n + 1)
        plt.imshow(delta_l)
        plt.title('Layer ' + str(n))
        plt.axis('off')
    plt.colorbar()
    plt.suptitle('Weight matrices variation')
    plt.show()

In [ ]:
# Construct a synthetic dataset

x_train = np.random.random((100, 4))
y_train = x_train

x_test = np.random.random((20, 4))
y_test = x_test

In [ ]:
# Compile and fit the model

model.compile(optimizer='adam',
              loss='mse',
              metrics=['acc'])

model.fit(x_train, y_train, epochs=50, verbose=False);

In [ ]:
# Retrieve weights and biases
W1_layers = get_weights(model)
b1_layers = get_biases(model)

In [ ]:
# Plot the variation

plt.figure(figsize=(8,8))
for n in range(3):
    delta_l = W1_layers[n] - W0_layers[n]
    print('Layer '+str(n)+': bias variation: ', np.linalg.norm(b1_layers[n] - b0_layers[n]))
    ax = plt.subplot(1,3,n+1)
    plt.imshow(delta_l)
    plt.title('Layer '+str(n))
    plt.axis('off')
plt.colorbar()
plt.suptitle('Weight matrices variation');

#### Freeze layers at build time

In [ ]:
# Count the trainable and non trainable variables before the freezing
from tensorflow.keras.layers import MaxPooling2D

inputs = Input(shape=(8, 8, 1), name='input_layer')
h = Conv2D(16, 3, activation='relu', name='conv2d_layer')(inputs)
h = MaxPooling2D(3, name='max_pool2d_layer')(h)
h = Flatten(name='flatten_layer')(h)
outputs = Dense(10, activation='softmax', name='softmax_layer')(h)

model = Model(inputs=inputs, outputs=outputs)

In [ ]:
# Display the number of trainable and non trainable variables before the freezing

print("\n Before freezing:\n\t Number of trainable variables: ", n_trainable_variables,
                         "\n\t Number of non trainable variables: ", n_non_trainable_variables)

In [ ]:
# Build the model
model = Sequential([
    Dense(4, input_shape=(4, ), activation='relu', kernel_initializer='random_uniform', 
          bias_initializer='ones', trainable=False),
    Dense(2, activation='relu', kernel_initializer='lecun_normal', bias_initializer='ones'),
    Dense(4, activation='softmax')
])

In [ ]:
# Count the trainable and non trainable variables after the freezing
n_trainable_variables = len(model.trainable_variables)
n_non_trainable_variables = len(model.non_trainable_variables)

# Display the number of trainable and non-trainable variables before freezing
print("Before freezing: \n\t Number of trainable variables: ", n_trainable_variables,
      "\n\t Number of non Trainable variables: ", n_non_trainable_variables)


In [ ]:
# Display the number of trainable and non trainable variables after the freezing

print("\n After freezing:\n\t Number of trainable variables: ", n_trainable_variables,
                         "\n\t Number of non trainable variables: ", n_non_trainable_variables)

In [ ]:
# Retrieve weights and biases
W0_layers = get_weights(model)
b0_layers = get_biases(model)

In [ ]:
# Compile and fit the model

model.compile(optimizer='adam',
              loss='mse',
              metrics=['acc'])

model.fit(x_train, y_train, epochs=50, verbose=False);

In [ ]:
# Retrieve weights and biases
W1_layers = get_weights(model)
b1_layers = get_biases(model)

In [ ]:
# Plot the variation
plot_delta_weights(W0_layers, W1_layers, b0_layers, b1_layers)

#### Freeze layers of a pre-built model

In [ ]:
# Count the trainable and non trainable variables before the freezing

print("\n Before freezing:\n\t Number of trainable variables: ", len(model.trainable_variables),
                         "\n\t Number of non trainable variables: ", len(model.non_trainable_variables))

In [ ]:
# Freeze the second layer
model.layers[1].trainable=False

In [ ]:
# Count the trainable and non trainable variables after the freezing

print("\n After freezing:\n\t Number of trainable variables: ", len(model.trainable_variables),
                        "\n\t Number of non trainable variables: ", len(model.non_trainable_variables))

In [ ]:
# Compile and fit the model

model.compile(optimizer='adam',
              loss='mse',
              metrics=['acc'])

model.fit(x_train, y_train, epochs=50, verbose=False);

In [ ]:
# Retrieve weights and biases
W2_layers = get_weights(model)
b2_layers = get_biases(model)

In [ ]:
# Plot the variation
plot_delta_weights(W1_layers, W2_layers, b1_layers, b2_layers)